In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [44]:
B, C, H, W = 2, 3, 4 , 4

x = torch.arange(0,B*C*H*W).view(B,C,H,W).float()
x

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]],

         [[32., 33., 34., 35.],
          [36., 37., 38., 39.],
          [40., 41., 42., 43.],
          [44., 45., 46., 47.]]],


        [[[48., 49., 50., 51.],
          [52., 53., 54., 55.],
          [56., 57., 58., 59.],
          [60., 61., 62., 63.]],

         [[64., 65., 66., 67.],
          [68., 69., 70., 71.],
          [72., 73., 74., 75.],
          [76., 77., 78., 79.]],

         [[80., 81., 82., 83.],
          [84., 85., 86., 87.],
          [88., 89., 90., 91.],
          [92., 93., 94., 95.]]]])

In [45]:
batchnorm = nn.BatchNorm2d(C)
batchnorm(x)

tensor([[[[-1.2889, -1.2480, -1.2071, -1.1662],
          [-1.1253, -1.0843, -1.0434, -1.0025],
          [-0.9616, -0.9207, -0.8798, -0.8388],
          [-0.7979, -0.7570, -0.7161, -0.6752]],

         [[-1.2889, -1.2480, -1.2071, -1.1662],
          [-1.1253, -1.0843, -1.0434, -1.0025],
          [-0.9616, -0.9207, -0.8798, -0.8388],
          [-0.7979, -0.7570, -0.7161, -0.6752]],

         [[-1.2889, -1.2480, -1.2071, -1.1662],
          [-1.1253, -1.0843, -1.0434, -1.0025],
          [-0.9616, -0.9207, -0.8798, -0.8388],
          [-0.7979, -0.7570, -0.7161, -0.6752]]],


        [[[ 0.6752,  0.7161,  0.7570,  0.7979],
          [ 0.8388,  0.8798,  0.9207,  0.9616],
          [ 1.0025,  1.0434,  1.0843,  1.1253],
          [ 1.1662,  1.2071,  1.2480,  1.2889]],

         [[ 0.6752,  0.7161,  0.7570,  0.7979],
          [ 0.8388,  0.8798,  0.9207,  0.9616],
          [ 1.0025,  1.0434,  1.0843,  1.1253],
          [ 1.1662,  1.2071,  1.2480,  1.2889]],

         [[ 0.6752,  0.7161,

In [46]:
batchnorm.running_mean, batchnorm.running_var

(tensor([3.1500, 4.7500, 6.3500]), tensor([62.5516, 62.5516, 62.5516]))

In [47]:
batchnorm.weight,batchnorm.bias

(Parameter containing:
 tensor([1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0.], requires_grad=True))

In [48]:
batchnorm.eps

1e-05

In [49]:
x.view(B, C, H* W ).var(dim=-1)

tensor([[22.6667, 22.6667, 22.6667],
        [22.6667, 22.6667, 22.6667]])

In [50]:
x.view(B, C, H* W ).mean(dim=-1)

tensor([[ 7.5000, 23.5000, 39.5000],
        [55.5000, 71.5000, 87.5000]])

In [51]:
x.var(correction=False,axis=1)

tensor([[[170.6667, 170.6667, 170.6667, 170.6667],
         [170.6667, 170.6667, 170.6667, 170.6667],
         [170.6667, 170.6667, 170.6667, 170.6667],
         [170.6667, 170.6667, 170.6667, 170.6667]],

        [[170.6667, 170.6667, 170.6667, 170.6667],
         [170.6667, 170.6667, 170.6667, 170.6667],
         [170.6667, 170.6667, 170.6667, 170.6667],
         [170.6667, 170.6667, 170.6667, 170.6667]]])

In [52]:
x.mean(axis=1)

tensor([[[16., 17., 18., 19.],
         [20., 21., 22., 23.],
         [24., 25., 26., 27.],
         [28., 29., 30., 31.]],

        [[64., 65., 66., 67.],
         [68., 69., 70., 71.],
         [72., 73., 74., 75.],
         [76., 77., 78., 79.]]])

In [94]:
torch.manual_seed(1)

norm = nn.BatchNorm2d(3,momentum=0.10)
d = torch.randn(2, 3, 2, 3)
d[0][0]

tensor([[-1.5256, -0.7502, -0.6540],
        [-1.6095, -0.1002, -0.6092]])

In [98]:
var = torch.split(d, 1, dim=1)[0].var(correction=False)
mean = torch.split(d, 1, dim=1)[0].mean()
mean,var
# (d[0][0][0][0] - mean) * ( var - 0.00005)**-0.5

(tensor(-0.3690), tensor(0.4563))

In [84]:
d = norm(d)
d[0][0][0]

tensor([-1.7121, -0.5643, -0.4219], grad_fn=<SelectBackward0>)

In [92]:
running_mean = (1 - norm.momentum) * 0 + norm.momentum * mean
running_var = (1 - norm.momentum) * 0 + norm.momentum * var
running_mean, running_var

(tensor(-0.0369), tensor(0.0456))

In [93]:
norm.running_mean[0], norm.running_var[0]

(tensor(-0.0369), tensor(0.9498))

In [79]:
norm.training = False
d = norm(d)
d[0][0][0]

tensor([-1.7189, -0.5412, -0.3950], grad_fn=<SelectBackward0>)

In [57]:
d = norm(d)
d[0][0][0]

tensor([-1.7434, -0.4675, -0.3091], grad_fn=<SelectBackward0>)

In [58]:
import torch

class ManualBatchNorm1d(torch.nn.Module):
    def __init__(self, num_features, momentum=0.1, eps=1e-5):
        super(ManualBatchNorm1d, self).__init__()
        self.num_features = num_features
        self.momentum = momentum
        self.eps = eps

        # Initialize parameters
        self.weight = torch.nn.Parameter(torch.ones(num_features))
        self.bias = torch.nn.Parameter(torch.zeros(num_features))

        # Initialize running mean and variance
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        if self.training:
            # Compute mean and variance over the batch dimension
            batch_mean = x.mean(dim=0)
            batch_var = x.var(dim=0, unbiased=False)

            # Update running mean and variance with momentum
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * batch_mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * batch_var

            # Normalize input using batch statistics
            x_normalized = (x - batch_mean) / torch.sqrt(batch_var + self.eps)
        else:
            # Normalize input using running statistics
            x_normalized = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)

        # Scale and shift normalized input
        return x_normalized * self.weight + self.bias


# Create a manual BatchNorm layer
num_features = 4
batchnorm = ManualBatchNorm1d(num_features)
batchnorm2 = nn.BatchNorm1d(num_features)

# Create a tensor of shape (batch_size, num_features)
batch_size = 2
input_tensor = torch.randn(batch_size, num_features)
print(input_tensor)

# Forward pass through the manual BatchNorm layer
output_tensor = batchnorm(input_tensor)
output_tensor1 = batchnorm2(input_tensor)

print("Output tensor after manual BatchNorm:", output_tensor)
print("Output tensor after manual BatchNorm:", output_tensor1)


tensor([[ 1.3790,  2.5286,  0.4107, -0.9880],
        [-0.9081,  0.5423,  0.1103, -2.2590]])
Output tensor after manual BatchNorm: tensor([[ 1.0000,  1.0000,  0.9998,  1.0000],
        [-1.0000, -1.0000, -0.9998, -1.0000]], grad_fn=<AddBackward0>)
Output tensor after manual BatchNorm: tensor([[ 1.0000,  1.0000,  0.9998,  1.0000],
        [-1.0000, -1.0000, -0.9998, -1.0000]],
       grad_fn=<NativeBatchNormBackward0>)
